# Imports & Data

In [1]:
import os

from datetime import datetime
from pathlib import Path

from tkinter import *
from tkinter.ttk import *
from tkinter import filedialog as fd
from tkinter import messagebox 
from tkinter.filedialog import askdirectory, askopenfile, askopenfilename

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import seaborn as sns

import math

In [49]:
def getFilename():
    filename = fd.askopenfilename()
    return Path(filename)

In [108]:
def single_path_split(filename):
    site = str(files[n]).split("\\", 7)
    site = site[7].split(" ")[0]
    correlate = str(files[n]).split("\\", 9)
    correlate = correlate[8]
    title = str(files[n]).split("Data\\", 1)
    title = title[1]
    title = title[:-4]
    title_split = title.split('_')
    reader_num = title_split[1]
    u_num = title_split[2]
    date = title_split[3]
    date = datetime.strptime(date, "%d%m%Y")
    date = date.strftime("%d-%m-%y")

    return title, site, correlate, reader_num, u_num, date

In [63]:
def getPath():
    root = Tk()
    root.withdraw()
    InputPath = askdirectory(title="Choose Directory")
    root.destroy()
    p = Path(InputPath)
    
    files = []

    for item in p.glob('**/*'):
        if item.suffix in ['.txt']:
            name = item.name
            dname = name
            dname = dname[:-8]
            files.append(str(p) + "\\" + str(dname) + "\\" + "Data" + "\\" + str(name))
            
    return files

In [64]:
files = getPath()

In [107]:
def multi_path_split(files):
    plot_titles = []
    reader = []
    u_number = []
    date_list = []
    site_list = []
    correlate_list = []
    
    for n in range(0, len(files), 2):
        site = str(files[n]).split("\\", 7)
        site = site[7].split(" ")[0]
        correlate = str(files[n]).split("\\", 9)
        correlate = correlate[8]
        title = str(files[n]).split("Data\\", 1)
        title = title[1]
        title = title[:-4]
        title_split = title.split('_')
        reader_num = title_split[1]
        u_num = title_split[2]
        date = title_split[3]
        date = datetime.strptime(date, "%d%m%Y")
        date = date.strftime("%d-%m-%y")
        plot_titles.append(title)
        reader.append(reader_num)
        u_number.append(u_num)
        date_list.append(date)
        site_list.append(site)
        correlate_list.append(correlate)
        
    path_df = pd.DataFrame(
        {"reader": reader,
         "u number": u_number,
         "date": date_list,
         "site": site_list,
         "correlate": correlate_list   
        }) 
    
    return plot_titles, site, date, correlate_list, reader, u_number

In [106]:
multi_path_split(files)

(['MOD_GZ0010719HMX_U1000_22022021_130215_Raw',
  'MOD_GZ0010719HMX_U1001_27052020_123000_Raw',
  'MOD_GZ0010719HMX_U1005_23022021_122712_Raw',
  'MOD_GZ0010719HMX_U1006_23022021_123120_Raw',
  'MOD_GZ0010719HMX_U1007_23022021_123557_Raw',
  'MOD_GZ0010719HMX_U1008_23022021_123947_Raw',
  'MOD_GZ0010719HMX_U1009_23022021_124323_Raw',
  'MOD_GZ0010719HMX_U1010_23022021_124654_Raw',
  'MOD_GZ0010719HMX_U1011_23022021_125207_Raw',
  'MOD_GZ0010719HMX_U1014_29052020_100122_Raw',
  'MOD_GZ0010719HMX_U1020_30052020_113201_Raw',
  'MOD_GZ0010719HMX_U1021_30052020_113509_Raw',
  'MOD_GZ0010719HMX_U1041_04062020_123320_Raw',
  'MOD_GZ0010719HMX_U1046_05062020_122737_Raw',
  'MOD_GZ0010719HMX_U1049_03032021_122457_Raw',
  'MOD_GZ0010719HMX_U1050_03032021_122843_Raw',
  'MOD_GZ0010719HMX_U1051_03032021_123346_Raw',
  'MOD_GZ0010719HMX_U1052_03032021_125102_Raw',
  'MOD_GZ0010719HMX_U1059_08062020_131515_Raw',
  'MOD_GZ0010719HMX_U1060_08062020_131821_Raw',
  'MOD_GZ0010719HMX_U1071_09032021_10300

In [155]:
# add the info for other columns in here so they can get passed along through the other functions

def single_cycle_plot(data_file):
    
    df = pd.read_csv(data_file, names=['y'])
    df = df[0:21000]

    sns.set_style("dark")
    plt.figure(figsize = (12, 5))
    plt.title(f'All Cycles:')
    
    plt.plot(df)
    
    return df

In [6]:
def indiv_cycles(df):

    data_list = df['y'].to_list()
    n = 700
    cycle_arrays = [data_list[i: i + n] for i in range(0, len(data_list), n)]
    cycle_arrays = np.array(cycle_arrays)

#     plot_title = graph_title(data_file)
    
    sns.set_style("dark")
    plt.figure(figsize = (12, 5))
    plt.title('Stacked Cycles')

    for x in range(0, len(cycle_arrays)):
        plt.plot(cycle_arrays[x])
        
    return cycle_arrays

In [7]:
def zero_pinned(cycle_arrays):
    
    pinned_arrays = np.array(cycle_arrays)
    
    sns.set_style("dark")
    plt.figure(figsize = (12, 5))
    plt.title('Zero Pinned Cycles')

    for x in range(0, len(pinned_arrays)):
        pinned_arrays[x] = pinned_arrays[x] - pinned_arrays[x][0]
        plt.plot(pinned_arrays[x])

    return pinned_arrays

In [127]:
def double_pinned(pinned_arrays):
    
    copy_pinned = pinned_arrays.copy()
    x_list = copy_pinned
    
    for i in range(0, len(pinned_arrays)):
        for j in range(0, 700):
            x_list[i][j] = j + 1

    new_cycle_array_x = x_list
    new_cycle_array_y = pinned_arrays

    for n in range(0, len(new_cycle_array_y)):

        new_x = []
        new_y = []
        new_xp = []
        new_yp = []

        '''For loop to pin each cycle to the origin and create list of the last value in each cycle'''

        new_y = pinned_arrays[n]
        end_yp = new_y[699]

        for i in range(0, len(new_y)):
            new_x.append(i)
            new_xp.append(i)
            new_yp.append(i)

        '''While loop to adjust cycle depending on its last value...'''

        '''Clockwise adjustment if cycle ends above 0 (postive value)'''

        rad = new_y[699]/700

        for i in range(0, len(new_y)):
            xp =  new_x[i] * math.cos(rad) + new_y[i] * math.sin(rad)
            yp = -new_x[i] * math.sin(rad) + new_y[i] * math.cos(rad)
            new_xp[i] = xp
            new_yp[i] = yp
            end_yp = yp

        new_cycle_array_x[n] = new_xp 
        new_cycle_array_y[n] = new_yp

    sns.set_style("dark")
    plt.figure(figsize = (12, 5))
    plt.title('Double Pinned Cycles')
        
    for n in range(0, len(new_cycle_array_y)):
        plt.plot(new_cycle_array_x[n], new_cycle_array_y[n])
            
    return new_cycle_array_x, new_cycle_array_y

In [153]:
def slope(double_pinned_arrays_x, double_pinned_arrays_y):
    rising_slope = []
    for n in range(0, len(double_pinned_arrays_y)):
        slopes = (double_pinned_arrays_y[n][200] - double_pinned_arrays_y[n][100]) / (double_pinned_arrays_x[n][200] - double_pinned_arrays_x[n][100])
        rising_slope.append(slopes)
        
    return rising_slope

In [9]:
def mean_height(double_pinned_arrays_y):
    
    plateau_avgs = []
    for n in range(0, len(double_pinned_arrays_y)):
        plateau_vals = double_pinned_arrays_y[n][200:380:1]
        plateau_avgs.append(sum(plateau_vals) / len(plateau_vals))
    
    return plateau_avgs

In [151]:
# TURN LAMBDA INTO FUNCTION TO MAKE THE BELOW MORE PYTHONIC

def width_stats(double_pinned_arrays_y):
    rising_edges_15 = []
    for n in range(0, len(double_pinned_arrays_y)):
        rising_edges_15.append(list(map(lambda x: x > (plateau_avgs[n] * .15), double_pinned_arrays_y[n])).index(True))

    falling_edges_list_15 = []
    for n in range(0, len(double_pinned_arrays_y)):
        falling_edges_list_15.append(list(map(lambda x: x > (plateau_avgs[n] * .15), double_pinned_arrays_y[n][::-1])).index(True))
        falling_edges_15 = [700 - i for i in falling_edges_list_15]

    width_15 = []
    zip_object = zip(rising_edges_15, falling_edges_15)
    for rising_edges_15_i, falling_edges_15_i in zip_object:
        width_15.append(falling_edges_15_i - rising_edges_15_i)
        
    rising_edges_30 = []
    for n in range(0, len(double_pinned_arrays_y)):
        rising_edges_30.append(list(map(lambda x: x > (plateau_avgs[n] * .3), double_pinned_arrays_y[n])).index(True))

    falling_edges_list_30 = []
    for n in range(0, len(double_pinned_arrays_y)):
        falling_edges_list_30.append(list(map(lambda x: x > (plateau_avgs[n] * .3), double_pinned_arrays_y[n][::-1])).index(True))
        falling_edges_30 = [700 - i for i in falling_edges_list_30]

    width_30 = []
    zip_object = zip(rising_edges_30, falling_edges_30)
    for rising_edges_30_i, falling_edges_30_i in zip_object:
        width_30.append(falling_edges_30_i - rising_edges_30_i)

    rising_edges_45 = []
    for n in range(0, len(double_pinned_arrays_y)):
        rising_edges_45.append(list(map(lambda x: x > (plateau_avgs[n] * .45), double_pinned_arrays_y[n])).index(True))

    falling_edges_list_45 = []
    for n in range(0, len(double_pinned_arrays_y)):
        falling_edges_list_45.append(list(map(lambda x: x > (plateau_avgs[n] * .45), double_pinned_arrays_y[n][::-1])).index(True))
        falling_edges_45 = [700 - i for i in falling_edges_list_45]

    width_45 = []
    zip_object = zip(rising_edges_45, falling_edges_45)
    for rising_edges_45_i, falling_edges_45_i in zip_object:
        width_45.append(falling_edges_45_i - rising_edges_45_i)    

    rising_edges_50 = []
    for n in range(0, len(double_pinned_arrays_y)):
        rising_edges_50.append(list(map(lambda x: x > (plateau_avgs[n] * .5), double_pinned_arrays_y[n])).index(True))

    falling_edges_list_50 = []
    for n in range(0, len(double_pinned_arrays_y)):
        falling_edges_list_50.append(list(map(lambda x: x > (plateau_avgs[n] * .5), double_pinned_arrays_y[n][::-1])).index(True))
        falling_edges_50 = [700 - i for i in falling_edges_list_50]

    width_50 = []
    zip_object = zip(rising_edges_50, falling_edges_50)
    for rising_edges_50_i, falling_edges_50_i in zip_object:
        width_50.append(falling_edges_50_i - rising_edges_50_i)
        
    rising_edges_65 = []
    for n in range(0, len(double_pinned_arrays_y)):
        rising_edges_65.append(list(map(lambda x: x > (plateau_avgs[n] * .65), double_pinned_arrays_y[n])).index(True))
    
    falling_edges_list_65 = []
    for n in range(0, len(double_pinned_arrays_y)):
        falling_edges_list_65.append(list(map(lambda x: x > (plateau_avgs[n] * .65), double_pinned_arrays_y[n][::-1])).index(True))
        falling_edges_65 = [700 - i for i in falling_edges_list_65]

    width_65 = []
    zip_object = zip(rising_edges_65, falling_edges_65)
    for rising_edges_65_i, falling_edges_65_i in zip_object:
        width_65.append(falling_edges_65_i - rising_edges_65_i)    
        
    rising_edges_80 = []
    for n in range(0, len(double_pinned_arrays_y)):
        rising_edges_80.append(list(map(lambda x: x > (plateau_avgs[n] * .80), double_pinned_arrays_y[n])).index(True))
    
    falling_edges_list_80 = []
    for n in range(0, len(double_pinned_arrays_y)):
        falling_edges_list_80.append(list(map(lambda x: x > (plateau_avgs[n] * .80), double_pinned_arrays_y[n][::-1])).index(True))
        falling_edges_80 = [700 - i for i in falling_edges_list_65]

    width_80= []
    zip_object = zip(rising_edges_80, falling_edges_80)
    for rising_edges_80_i, falling_edges_80_i in zip_object:
        width_80.append(falling_edges_80_i - rising_edges_80_i)  

    rising_slope = slope(double_pinned_arrays_x, double_pinned_arrays_y)
        
    metrics_df = pd.DataFrame(
        {'rising_edges_15': rising_edges_15,
         'width_15': width_15,
         'rising_edges_30': rising_edges_30,
         'width_30': width_30,
         'rising_edges_45': rising_edges_45,
         'width_45': width_45,
         'rising_edges_50': rising_edges_50,
         'width_50': width_50,
         'rising_edges_65': rising_edges_65,
         'width_65': width_65,
         'rising_edges_80': rising_edges_80,
         'width_80': width_80,
         'rising_slope': rising_slope
        })
    
    metrics_df['cycle_number'] = metrics_df.index

    metrics_df.loc['mean'] = metrics_df.mean()
    
    metrics_df[["cycle_number", "rising_edges_15", "width_15", "rising_edges_30", "width_30", "rising_edges_45", "width_45", "rising_edges_50", "width_50", "rising_edges_65", "width_65", "rising_edges_80", "width_80"]] = metrics_df[["cycle_number", "rising_edges_15", "width_15", "rising_edges_30", "width_30", "rising_edges_45", "width_45", "rising_edges_50", "width_50", "rising_edges_65", "width_65", "rising_edges_80", "width_80"]].astype(int)    
    
    cols = metrics_df.columns.tolist()
    cols = cols[-1:] + cols[:-1]
    metrics_df = metrics_df[cols]
    
    metrics_df.to_csv('metrics_df_new.csv', index=False)
    
    return print(metrics_df)

# Main Loop

In [158]:
# create pop window that asks if user wants to do a recursive loop or not
state_1 = messagebox.askyesno("Manual or Recursive loop?","Recursive loop?")

# if user selects yes, allow user to choose directory and then output the selected directory 
if state_1 == True:
    print("Recursive selection")
    directory = getPath()
    mode = "recursive"
    
# keep 2 for step size to skip over time text files    
    for i in range(0, len(directory), 2):
#         print(files[i])
        data_file = directory[i]
        df = single_cycle_plot(data_file)
        cycle_arrays = indiv_cycles(df)
        pinned_arrays = zero_pinned(cycle_arrays)
        double_pinned_arrays = double_pinned(pinned_arrays)
        
        double_pinned_arrays_x = double_pinned_arrays[0]
        double_pinned_arrays_y = double_pinned_arrays[1]
        
        plateau_avgs = mean_height(double_pinned_arrays_y)
        rising_slope = slope(double_pinned_arrays_x, double_pinned_arrays_y)
        width_metrics = width_stats(double_pinned_arrays_y)
        # comment out if do the full 
#         stop = messagebox.askyesno("Pause?","Continue with next file?")
#         if stop == False:
#             break
        
    
    
# if user selects no, go into manual mode
else:
    print("Manual selection")
    state_1 = False
    mode = "manual"

    # once in manual mode, allow user to select a file and then ask if they want to select another
    while state_1 == False:
        data_file = getFilename()

        df = single_cycle_plot(data_file)
        cycle_arrays = indiv_cycles(df)
        pinned_arrays = zero_pinned(cycle_arrays)
        double_pinned_arrays = double_pinned(pinned_arrays)
        
        double_pinned_arrays_x = double_pinned_arrays[0]
        double_pinned_arrays_y = double_pinned_arrays[1]
        
        plateau_avgs = mean_height(double_pinned_arrays_y)
        rising_slope = slope(double_pinned_arrays_x, double_pinned_arrays_y)
        width_metrics = width_stats(double_pinned_arrays_y)
        
        another_file = messagebox.askyesno("Another file?","Continue with next file?")

        if another_file == False:
            state_1 = True


Manual selection


PermissionError: [Errno 13] Permission denied: '.'

In [ ]:
sns.set_style('darkgrid')
ax = sns.histplot(plateau_avgs)
ax.locator_params(axis='y', integer=True);

In [ ]:
sns.set_style('darkgrid')
ax = sns.boxplot(x=plateau_avgs);
# ax.xaxis.set_major_locator(ticker.MultipleLocator())
# ax.xaxis.set_major_formatter(ticker.ScalarFormatter());

This 'for loop' captures the mean value of each cycle's plateau.

In [ ]:
mean = sum(plateau_avgs) / len(plateau_avgs)
variance = sum([((x - mean) ** 2) for x in plateau_avgs]) / len(plateau_avgs)
std_dev = variance ** 0.5
std_dev

For this sample, there is a very small standard deviation, demonstrating a tight range during this portion of each cycle.

In [ ]:
sns.set_style('darkgrid')
ax = sns.histplot(width_15, discrete=True)
ax.locator_params(axis='x', integer=True)
ax.locator_params(axis='y', integer=True);

In [ ]:
sns.set_style('darkgrid')
ax = sns.boxplot(x=width_15);

In [ ]:
sns.set_style('darkgrid')
ax = sns.histplot(width_30, discrete=True)
ax.locator_params(axis='x', integer=True)
ax.locator_params(axis='y', integer=True);

In [ ]:
sns.set_style('darkgrid')
ax = sns.boxplot(x=width_30)
ax.xaxis.set_major_locator(ticker.MultipleLocator())
ax.xaxis.set_major_formatter(ticker.ScalarFormatter());

In [ ]:
sns.set_style('darkgrid')
ax = sns.histplot(width_45, discrete=True)
ax.locator_params(axis='x', integer=True)
ax.locator_params(axis='y', integer=True);

In [ ]:
sns.set_style('darkgrid')
ax = sns.boxplot(x=width_45)
ax.xaxis.set_major_locator(ticker.MultipleLocator())
ax.xaxis.set_major_formatter(ticker.ScalarFormatter());

In [ ]:
sns.set_style('darkgrid')
ax = sns.histplot(width_50, discrete=True)
ax.locator_params(axis='x', integer=True)
ax.locator_params(axis='y', integer=True);

In [ ]:
sns.set_style('darkgrid')
ax = sns.boxplot(x=width_50)
ax.xaxis.set_major_locator(ticker.MultipleLocator())
ax.xaxis.set_major_formatter(ticker.ScalarFormatter());

In [ ]:
sns.set_style('darkgrid')
ax = sns.histplot(width_65, discrete=True)
ax.locator_params(axis='x', integer=True)
ax.locator_params(axis='y', integer=True);

In [ ]:
sns.set_style('darkgrid')
ax = sns.boxplot(x=width_65)
ax.xaxis.set_major_locator(ticker.MultipleLocator())
ax.xaxis.set_major_formatter(ticker.ScalarFormatter());

In [ ]:
sns.set_style('darkgrid')
ax = sns.histplot(width_80, discrete=True)
ax.locator_params(axis='x', integer=True)
ax.locator_params(axis='y', integer=True);

In [ ]:
sns.set_style('darkgrid')
ax = sns.boxplot(x=width_80)
ax.xaxis.set_major_locator(ticker.MultipleLocator())
ax.xaxis.set_major_formatter(ticker.ScalarFormatter());

In [ ]:
averages

The averages array (see above) contains the mean height of each sample as well as the width measurements at 15%, 30%, 45%, 50%, 65%, and 80%.

In [ ]:
overall_avg = []
for i in averages:
    overall_avg.append(sum(i) / len(i))
overall_avg

In [ ]:
# export all metrics into a csv 

# first create a dictionary of the width metrics
width_metrics = {'mean_height':plateau_avgs, 'width_15': width_15, 'width_30': width_30, 'width_45': width_45, 
                 'width_50': width_50, 'width_65':width_65, 'width_80':width_80}

In [ ]:
width_metrics = pd.DataFrame(width_metrics)
width_metrics

In [ ]:
width_metrics.loc['mean'] = width_metrics.mean()
width_metrics[["width_15", "width_30", "width_45", "width_50", "width_65", "width_80"]] = width_metrics[["width_15", "width_30", "width_45", 
                                                                                                         "width_50", "width_65", "width_80"]].astype(int)
width_metrics

In [ ]:
# consolidate all of the metrics above into a single function and output like below

width_metrics.to_csv('width_metrics.csv')

# OLD CODE TO DELETE

Pseudo Code for Recursive Scan of Directory and then apply Visual/Metric building function to those files:     
1) Go into a given directory and build a list of files   
    Use tkinter to offer option to select one file at a time (manual option) or to go through entire directory (recursive option)  
    Like Daren did in his code below, this will require a function that uses for loops that work off the answer from message box  
    Also requires a pause option to review after processing a data file    
    
2) Compile the code that builds the plots and creates the width metrics for the cycles into a single function (or maybe a few shorter ones)   
    Apply those functions to the list of files created above   
    Output the metrics to a csv or txt file    
    Output plots to pngs  

In [ ]:
df = pd.read_csv(r'../Data/U44_Raw.txt', names=['y'])
df.to_csv('../Data/test_data_1.csv')

# Exploratory Data Analysis

In [ ]:
df.head(10)

In [ ]:
df.shape

In [ ]:
sns.set_style("dark")
plt.figure(figsize = (12, 5))

cycle = df.iloc[:700]

plt.plot(cycle)

In [ ]:
sns.set_style("dark")
plt.figure(figsize = (15, 5))

plt.plot(df)
plt.title("All Cycles", fontsize=20,fontweight=0, color='orange');

In [ ]:
data_list = df['y'].to_list()
cycle_arrays = np.array_split(data_list, 30)
len(cycle_arrays)

In [ ]:
n = 700
cycle_arrays = [data_list[i: i +n] for i in range(0, len(data_list), n)]

len(cycle_arrays)

In [ ]:
cycle_arrays

In [ ]:
df['x'] = range(21000)
x_list = df['x'].to_list()
x_arrays = np.array_split(x_list, 30)

In [ ]:
df['x'] = range(0,len(df))
df['x']
x_list = df['x'].to_list()

n = 700
x_list = [x_list[i: i + n] for i in range(0, len(x_list), n)]
x_list

In [ ]:
sns.set_style("dark")
plt.figure(figsize = (12, 5))

cycles_mean = np.mean(cycle_arrays, axis=0)

plt.plot(cycles_mean)
plt.title("Mean of Cycles (No Adjustment)", fontsize=20,fontweight=0, color='orange');

Here is the mean of all signals 30 signals, captured in its own plot. Now, let's add the mean to the rest of the plots to highlight the noise in the signals.

In [ ]:
cycle_arrays.append(cycles_mean)
cycle_arrays

In [ ]:
len(cycle_arrays)

In [ ]:
sns.set_style("dark")
plt.figure(figsize = (12, 5))

for i in range(len(cycle_arrays)):
    plt.plot(cycle_arrays[i])
    
plt.title("All Cycles (No Adjustment)", fontsize=20,fontweight=0, color='orange');

In [ ]:
sns.set_style("dark")
plt.figure(figsize = (12, 5))

for i in range(len(cycle_arrays)):
    plt.plot(cycle_arrays[i], marker='', color='grey', linewidth=4, alpha=0.4)
    
plt.plot(cycle_arrays[-1], marker='', color='orange', linewidth=4, alpha=0.7)
plt.title("All Cycles (Mean Cycle Highlighted)", fontsize=20,fontweight=0, color='orange');

In [ ]:
plt.figure(figsize = (15, 5))

for i in range(len(cycle_arrays)):
    cycle_arrays[i] = cycle_arrays[i] - cycle_arrays[i][0]
    plt.plot(cycle_arrays[i])
    
plt.title("All Cycles (Origin Pinned)", fontsize=20,fontweight=0, color='orange');

In [ ]:
plt.figure(figsize = (15, 5))

for i in range(len(cycle_arrays)):
    cycle_arrays[i] = cycle_arrays[i] - cycle_arrays[i][0]
    plt.plot(cycle_arrays[i], marker='', color='grey', linewidth=4, alpha=0.4)
    
plt.plot(cycle_arrays[-1], marker='', color='orange', linewidth=2, alpha=0.7)    
plt.title("All Cycles (Origin Pinned & Mean Highlighted)", fontsize=20, fontweight=0, color='orange');

Pseudo-code for the function below:

Main objectives:  

Iterate through each of the 30 cycles of a raw data sample and make each one end at zero.  

Create a while loop that determines if the cycle needs to be adjusted clockwise, counter-clockwise, or not at all.  

Plot each of the adjusted cycles on the same figure.

In [ ]:
cycle_arrays.pop()

In [ ]:
len(cycle_arrays)

## With average shown

In [115]:
# turn this into a function, save plot output to png

# def adjust_cycles(df):
# df = pd.read_csv(r'../Data/U44_Raw.txt', names=['y']) >>> need to do this programmatically instead of hardcoded

new_cycle_array_x = np.array_split(x_list, 30)
new_cycle_array_y = np.array_split(data_list, 30)

for n in range(0, len(cycle_arrays)):
    cycle_arrays[n] = cycle_arrays[n] - cycle_arrays[n][0]
#     print('n', n)

# for i in range(0, len(cycle_arrays[n])):  
#     print("\n>>> Starting rotation correction")

    # Empty lists 
    new_x = []
    new_y = []
    new_xp = []
    new_yp = []

    angle_step = 0

    rad = angle_step * math.pi/180

    '''For loop to pin each cycle to the origin and create list of the last value in each cycle'''

    new_y = cycle_arrays[n]
    end_yp = new_y[699]

    for i in range(0, len(new_y)):
        new_x.append(i)
        new_xp.append(i)
        new_yp.append(i)

    '''While loop to adjust cycle depending on its last value...'''

    '''Clockwise adjustment if cycle ends above 0 (postive value)'''

    if end_yp > 0:
#         print("Signal correction needed is clockwise")
        while end_yp > 0.0:
            for i in range(0, len(new_y)):
                xp =  new_x[i] * math.cos(rad) + new_y[i] * math.sin(rad)
                yp = -new_x[i] * math.sin(rad) + new_y[i] * math.cos(rad)
                new_xp[i] = xp
                new_yp[i] = yp
                end_yp = yp

            angle_step = angle_step + 0.0000001
            rad = angle_step * math.pi/180
#         print(">>>Cycle Rotated Clockwise")
#         print(">>>beginning - last value in new_y:", round(new_y[698], 6))
#         print(">>>ending end_yp:", round(end_yp, 6))
#         print(">>>angle:", round(angle_step, 6))


    elif end_yp < 0:
#         print("Signal correction needed is counter-clockwise")
        while end_yp < 0.0:
            for i in range(0, len(new_y)):
                xp =  new_x[i] * math.cos(rad) + new_y[i] * math.sin(rad)
                yp = -new_x[i] * math.sin(rad) + new_y[i] * math.cos(rad)
                new_xp[i] = xp
                new_yp[i] = yp
                end_yp = yp

            angle_step = angle_step - 0.0000001
            rad = angle_step * math.pi/180
#         print(">>>Cycle Rotated Counter-Clockwise")
#         print(">>>beginning - last value in new_y:", round(new_y[698], 6))
#         print(">>>ending end_yp:", round(end_yp, 6))
#         print(">>>angle:", round(angle_step, 6))

    else:
#         print(">>>No Cycle Rotation Needed")  
        new_xp = new_x
        new_yp = new_y

    new_cycle_array_x[n] = new_xp 
    new_cycle_array_y[n] = new_yp
#     print(n)
#     print('new cycle array x[n]', new_cycle_array_x[n][0], new_xp[0])
#     print('new cycle array y[n]', new_cycle_array_y[n][0], new_yp[0])
    
    sns.set_style("dark")
    plt.rcParams["figure.figsize"] = (20,10)

    plt.plot(new_cycle_array_x[n], new_cycle_array_y[n], marker='', color='grey', linewidth=4, alpha=0.4)

plt.plot(new_cycle_array_y[-1], marker='', color='orange', linewidth=2, alpha=0.7) 

plt.savefig('adjusted_cycles_highlighted_mean.png')

NameError: name 'x_list' is not defined

In [ ]:
# save output to png

sns.set_style("dark")

for n in range(0, len(cycle_arrays)):
    xp = []
    for i in range(0, len(new_cycle_array_x[n])):
        xp.append(new_cycle_array_x[n][i] + 700 * n)

#     print('new cycle array x[n]', new_cycle_array_x[n][0], new_xp[0])
#     print('new cycle array y[n] 2', new_cycle_array_y[n][0], new_yp[0])
    plt.plot(xp, new_cycle_array_y[n])

plt.savefig('adjusted_cycles_connected.png')

In [ ]:
# for every value of y at given index in rising list, find the corresponding value of x
# for every value of y at given index in falling list, find the corresponding value of x
# subtract those two values of x to find width metric

## Test

In [ ]:
# what index number is the first value to be higher than 0.02 in the rising phase of each cycle?
rising_edges = []
for n in range(0, len(cycle_arrays)):
    rising_edges.append(list(map(lambda x: x > 0.02, new_cycle_array_y[n])).index(True))
rising_edges

In [ ]:
Counter(rising_edges)

In [ ]:
falling_edge_list = []

for n in range(0, len(cycle_arrays)):
    falling_edge_list.append(list(map(lambda x: x > 0.02, new_cycle_array_y[n][::-1])).index(True))
    falling_edges = [700 - i for i in falling_edge_list]

falling_edges

In [ ]:
Counter(falling_edges)

In [ ]:
xp_test = []
for i in range(0, len(new_cycle_array_x[n])):
    xp_test.append(new_cycle_array_x[n][i])

In [ ]:
xp_test

In [ ]:
xp_test[434] - xp_test[131]

is the decimal statistically relevant?

In [ ]:
width = []

zip_object = zip(rising_edges, falling_edges)
for rising_edges_i, falling_edges_i in zip_object:
    width.append(falling_edges_i - rising_edges_i)

print(width)

In [ ]:
Counter(width)

In [ ]:
mean = sum(width) / len(width)
variance = sum([((x - mean) ** 2) for x in width]) / len(width)
std_dev = variance ** 0.5
std_dev

In [ ]:
sns.set_style('darkgrid')
sns.histplot(width, discrete=True);

In [ ]:
sns.set_style('darkgrid')
ax = sns.boxplot(width)
ax.xaxis.set_major_locator(ticker.MultipleLocator())
ax.xaxis.set_major_formatter(ticker.ScalarFormatter())

In [ ]:
plt.plot(new_cycle_array_y[0])
plt.plot(new_cycle_array_y[1])
plt.plot(new_cycle_array_y[2])